# date_time

In [ ]:
open rust_operators
open sm'_operators

()



In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::DateTime<$0>\")>] type chrono_DateTime<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::Local\")>] type chrono_Local = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::NaiveDateTime\")>] type chrono_NaiveDateTime = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::Utc\")>] type chrono_Utc = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::time::Duration\")>] type std_time_Duration = class end"

()



In [ ]:
nominal ticks = i64
nominal duration = $"std_time_Duration"
nominal date_time = $"System.DateTime"
nominal date_time' t = $"chrono_DateTime<`t>"
nominal local = $"chrono_Local"
nominal naive_date_time = $"chrono_NaiveDateTime"
nominal utc = $"chrono_Utc"

()



## naive_utc

In [ ]:
inl naive_utc (date_time : date_time' utc) : naive_date_time =
    inl date_time = join date_time
    !\($'"!date_time.naive_utc()"')

()



## to_local

In [ ]:
inl to_local (date_time : date_time' utc) : date_time' local =
    inl naive_date_time = date_time |> naive_utc
    !\($'"chrono::offset::TimeZone::from_utc_datetime(&chrono::Local, &!naive_date_time)"')

()



## from_timestamp'

In [ ]:
inl from_timestamp' forall t {number; int}. (timestamp : t) : option (date_time' utc) =
    inl timestamp = join timestamp
    inl result : optionm'.option' (date_time' utc) =
        !\($'"chrono::DateTime::from_timestamp_micros(!timestamp / 1000i64)"')
    result |> optionm'.unbox

()



## format'

In [ ]:
inl format' (format : string) (date_time : date_time' utc) : sm'.std_string =
    inl format = #format
    inl date_time = join date_time
    !\($'"!date_time.format(!format).to_string()"')

()



## format''

In [ ]:
inl format'' (format : string) (date_time : date_time' _) : sm'.std_string =
    inl format = #format
    inl date_time = join date_time
    !\($'"!date_time.format(!format).to_string()"')

()



## format_timestamp

In [ ]:
inl format_timestamp (timestamp : sm'.std_string) =
    inl timestamp = join timestamp
    timestamp
    |> fun x => !\($'"!x.parse().unwrap()"') : i64
    |> from_timestamp'
    |> optionm.map fun x =>
        x
        |> to_local
        |> format'' "%Y-%m-%d %H:%M:%S"
        |> sm'.from_std_string
    |> resultm.from_option

()



## duration_from_millis

In [ ]:
inl duration_from_millis (ms : u64) : duration =
    inl ms = join ms
    !\($'"std::time::Duration::from_millis(!ms)"')

()



## get_environment_variable

In [ ]:
inl get_environment_variable (var : string) : string =
    $"System.Environment.GetEnvironmentVariable !var"

()



In [ ]:
// // test

inl test_guid () =
    guid.new_guid "FEDCBA98-7654-3210-FEDC-BA9876543210"

()



In [ ]:
type ticks_guid = guid.guid
type date_time_guid = guid.guid

()



## date_time_guid_from_date_time

In [ ]:
inl date_time_guid_from_date_time (guid : guid.guid) (date_time : date_time) =
    inl guid = guid |> sm'.obj_to_string
    inl prefix = $'!date_time.ToString "yyyyMMdd-HHmm-ssff-ffff-f"' : string
    $'`date_time_guid $"{!prefix}{!guid.[!prefix.Length..]}"' : date_time_guid

()



In [ ]:
// // test

date_time_guid_from_date_time (test_guid ()) $'System.DateTime.MinValue'
|> sm'.obj_to_string
|> _assert_eq' "00010101-0000-0000-0000-0a9876543210"

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : System.DateTime = System.DateTime.MinValue
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v1
    let v5 : string = v2.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    let v6 : System.Guid = System.Guid $"{v5}{v4.[v5.Length..]}"
    let v7 : (System.Guid -> string) = _.ToString()
    let v8 : string = v7 v6
    let v9 : string = $"%A{v8}"
    System.Console.WriteLine v9
    let v10 : string = "00010101-0000-0000-0000-0a9876543210"
    let v11 : bool = v8 = v10
    let v13 : bool =
        if v11 then
            true
        else
            method1(v11)
    let v14 : string = $"__expect / actual: %A{v8} / expected: %A{v10}"
    let v15 : bool = v13 = false
    if v15 then
        failwith<unit> v14
method0()

"00010101-0000-0000-0000-0a9876543210"


In [ ]:
// // test

date_time_guid_from_date_time (test_guid ()) $'System.DateTime.MaxValue'
|> sm'.obj_to_string
|> _assert_eq $'$"99991231-2359-5999-9999-9{(!test_guid() |> string).[^10..]}"'

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : System.DateTime = System.DateTime.MaxValue
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v1
    let v5 : string = v2.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    let v6 : System.Guid = System.Guid $"{v5}{v4.[v5.Length..]}"
    let v7 : (System.Guid -> string) = _.ToString()
    let v8 : string = v7 v6
    let v9 : (unit -> System.Guid) = closure0()
    let v10 : string = $"99991231-2359-5999-9999-9{(v9() |> string).[^10..]}"
    let v11 : string = $"%A{v8}"
    System.Console.WriteLine v11
    let v12 : bool = v8 = v10
    let v14 : bool =
        if v12 then
            true
        else
            method1(v12)
    let v15 : string 

In [ ]:
// // test

date_time_guid_from_date_time (test_guid ()) $'System.DateTime.UnixEpoch'
|> sm'.obj_to_string
|> _assert_eq $'$"19700101-0000-0000-0000-0{(!test_guid () |> string).[^10..]}"'

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : System.DateTime = System.DateTime.UnixEpoch
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v1
    let v5 : string = v2.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    let v6 : System.Guid = System.Guid $"{v5}{v4.[v5.Length..]}"
    let v7 : (System.Guid -> string) = _.ToString()
    let v8 : string = v7 v6
    let v9 : (unit -> System.Guid) = closure0()
    let v10 : string = $"19700101-0000-0000-0000-0{(v9 () |> string).[^10..]}"
    let v11 : string = $"%A{v8}"
    System.Console.WriteLine v11
    let v12 : bool = v8 = v10
    let v14 : bool =
        if v12 then
            true
        else
            method1(v12)
    let v15 : strin

## date_time_from_guid

In [ ]:
inl date_time_from_guid (date_time_guid : date_time_guid) =
    inl date_time_guid = date_time_guid |> sm'.obj_to_string
    inl sm'_replace = join sm'.replace
    run_target function
        | Fsharp => fun () => $'System.DateTime.ParseExact (!date_time_guid.[..24] |> !sm'_replace "-" "", "yyyyMMddHHmmssfffffff", null)' : date_time
        | Rust _ => fun () =>
            $'System.DateTime.Parse (!date_time_guid.[..24] |> !sm'_replace "-" "")' : date_time

()



In [ ]:
// // test

date_time_from_guid (guid.new_guid "00010101-0000-0000-0000-0a9876543210")
|> _assert_eq' $'System.DateTime.MinValue'

let rec closure2 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure1 (v0 : string) (v1 : string) : (string -> string) =
    closure2(v0, v1)
and closure0 () (v0 : string) : (string -> (string -> string)) =
    closure1(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure0()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "00010101-0000-0000-0000-0a9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (string -> (string -> (string -> string))) = method1()
    let mutable result = None
    #if FABLE_COMPILER_RUST && !WASM
    let v5 : System.DateTime = System.DateTime.Parse (v3.[..24] |> v4 "-" "")
    v5
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v6 : System.DateTime = System.DateTime.Parse (v3.[..24] |> v4 "-" "")
    v6
    #endif
    #if !FABLE_COMPILER && 

In [ ]:
// // test

date_time_from_guid (guid.new_guid $'$"99991231-2359-5999-9999-9{(!test_guid () |> string).[^10..]}"')
|> _assert_eq' $'System.DateTime.MaxValue'

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and closure3 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure2 (v0 : string) (v1 : string) : (string -> string) =
    closure3(v0, v1)
and closure1 () (v0 : string) : (string -> (string -> string)) =
    closure2(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure1()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> System.Guid) = closure0()
    let v1 : string = $"99991231-2359-5999-9999-9{(v0 () |> string).[^10..]}"
    let v2 : System.Guid = System.Guid v1 
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v2
    let v5 : (string -> (string -> (string -> string))) = method1()
    let mutable result = None
    #if FABLE_COMPILER_RUST && !WASM
    let v6 : System.DateTime = System.DateTi

In [ ]:
// // test

date_time_from_guid (guid.new_guid $'$"19700101-0000-0000-0000-0{(!test_guid () |> string).[^10..]}"')
|> _assert_eq' $'System.DateTime.UnixEpoch'

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and closure3 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure2 (v0 : string) (v1 : string) : (string -> string) =
    closure3(v0, v1)
and closure1 () (v0 : string) : (string -> (string -> string)) =
    closure2(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure1()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> System.Guid) = closure0()
    let v1 : string = $"19700101-0000-0000-0000-0{(v0 () |> string).[^10..]}"
    let v2 : System.Guid = System.Guid v1 
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v2
    let v5 : (string -> (string -> (string -> string))) = method1()
    let mutable result = None
    #if FABLE_COMPILER_RUST && !WASM
    let v6 : System.DateTime = System.DateTi

## ticks_guid_from_ticks

In [ ]:
inl ticks_guid_from_ticks (guid : guid.guid) (ticks : ticks) : ticks_guid =
    inl guid = guid |> sm'.obj_to_string
    inl ticks = ticks |> sm'.obj_to_string |> sm'.pad_left 18i32 '0'
    $'`ticks_guid $"{!ticks.[0..7]}-{!ticks.[8..11]}-{!ticks.[12..15]}-{!ticks.[16..17]}{!guid.[21..]}"'

()



In [ ]:
// // test

ticks_guid_from_ticks (test_guid ()) (ticks 0i64)
|> _assert_eq' (guid.new_guid "00000000-0000-0000-00dc-ba9876543210")

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (int64 -> string) = _.ToString()
    let v5 : string = v4 0L
    let v6 : string = v5.PadLeft (18, '0')
    let v7 : System.Guid = System.Guid $"{v6.[0..7]}-{v6.[8..11]}-{v6.[12..15]}-{v6.[16..17]}{v3.[21..]}"
    let v8 : string = "00000000-0000-0000-00dc-ba9876543210"
    let v9 : System.Guid = System.Guid v8 
    let v10 : string = $"%A{v7}"
    System.Console.WriteLine v10
    let v11 : bool = v7 = v9
    let v13 : bool =
        if v11 then
            true
        else
            method1(v11)
    let v14 : string = $"__expect / actual: %A{v7} / expected: %A{v9}"
    let v15 : bool = v13 = false
    if v15 then
        failwith<unit> v14
method0()

00000000-0000-0000-00dc-ba9876543210


In [ ]:
// // test

ticks_guid_from_ticks (test_guid ()) (ticks 999999999999999999i64)
|> _assert_eq' (guid.new_guid $'$"99999999-9999-9999-99dc-b{(!test_guid () |> string).[^10..]}"')

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (int64 -> string) = _.ToString()
    let v5 : string = v4 999999999999999999L
    let v6 : string = v5.PadLeft (18, '0')
    let v7 : System.Guid = System.Guid $"{v6.[0..7]}-{v6.[8..11]}-{v6.[12..15]}-{v6.[16..17]}{v3.[21..]}"
    let v8 : (unit -> System.Guid) = closure0()
    let v9 : string = $"99999999-9999-9999-99dc-b{(v8 () |> string).[^10..]}"
    let v10 : System.Guid = System.Guid v9 
    let v11 : string = $"%A{v7}"
    System.Console.WriteLine v11
    let v12 : bool = v7 = v10
    let v14 : bool =
        if v12 then
            true
        else
            method1(v12)


## ticks_from_guid

In [ ]:
inl ticks_from_guid (guid : date_time_guid) : ticks =
    inl guid = guid |> sm'.obj_to_string
    $'`i64 $"{!guid.[0..7]}{!guid.[9..12]}{!guid.[14..17]}{!guid.[19..20]}"'

()



In [ ]:
// // test

ticks_from_guid (guid.new_guid "00000000-0000-0000-00dc-ba9876543210")
|> _assert_eq (ticks 0)

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "00000000-0000-0000-00dc-ba9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : int64 = int64 $"{v3.[0..7]}{v3.[9..12]}{v3.[14..17]}{v3.[19..20]}"
    let v5 : string = $"%A{v4}"
    System.Console.WriteLine v5
    let v6 : bool = v4 = 0L
    let v8 : bool =
        if v6 then
            true
        else
            method1(v6)
    let v9 : string = $"__expect / actual: %A{v4} / expected: %A{0L}"
    let v10 : bool = v8 = false
    if v10 then
        failwith<unit> v9
method0()

0L


In [ ]:
// // test

ticks_from_guid (guid.new_guid $'$"99999999-9999-9999-99{(!test_guid () |> string).[^14..]}"')
|> _assert_eq (ticks 999999999999999999)

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> System.Guid) = closure0()
    let v1 : string = $"99999999-9999-9999-99{(v0 () |> string).[^14..]}"
    let v2 : System.Guid = System.Guid v1 
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v2
    let v5 : int64 = int64 $"{v4.[0..7]}{v4.[9..12]}{v4.[14..17]}{v4.[19..20]}"
    let v6 : string = $"%A{v5}"
    System.Console.WriteLine v6
    let v7 : bool = v5 = 999999999999999999L
    let v9 : bool =
        if v7 then
            true
        else
            method1(v7)
    let v10 : string = $"__expect / actual: %A{v5} / expected: %A{999999999999999999L}"
    let v11 : bool = v9 = false
    if v11 then
        failwith<unit> v10
method0()

999999999999999999L


## new_guid_from_date_time

In [ ]:
inl new_guid_from_date_time (date_time : date_time) =
    inl guid = guid.new_raw_guid ()
    date_time_guid_from_date_time guid date_time

()



In [ ]:
// // test

new_guid_from_date_time $'System.DateTime.UtcNow'
|> date_time_from_guid
|> fun date_time => $'(!date_time - System.DateTime.UtcNow).TotalSeconds' : f64 |> i32
|> _assert_eq 0

let rec closure2 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure1 (v0 : string) (v1 : string) : (string -> string) =
    closure2(v0, v1)
and closure0 () (v0 : string) : (string -> (string -> string)) =
    closure1(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure0()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : System.DateTime = System.DateTime.UtcNow
    let v1 : System.Guid = System.Guid.NewGuid ()
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : string = v0.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    let v5 : System.Guid = System.Guid $"{v4}{v3.[v4.Length..]}"
    let v6 : (System.Guid -> string) = _.ToString()
    let v7 : string = v6 v5
    let v8 : (string -> (string -> (string -> string))) = method1()
    let mutable result = None
    #if FABLE_COMPILER_RUST && !WASM
    let v9 : System.DateTime = System.DateTime.P

## new_guid_from_ticks

In [ ]:
inl new_guid_from_ticks (ticks : ticks) =
    inl guid = guid.new_raw_guid ()
    ticks_guid_from_ticks guid ticks

()



In [ ]:
// // test

new_guid_from_ticks $'System.DateTime.UtcNow.Ticks'
|> ticks_from_guid
|> fun ticks => $'(!ticks - System.DateTime.UtcNow.Ticks) / 100000L'
|> _assert_eq 0i64

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int64 = System.DateTime.UtcNow.Ticks
    let v1 : System.Guid = System.Guid.NewGuid ()
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (int64 -> string) = _.ToString()
    let v5 : string = v4 v0
    let v6 : string = v5.PadLeft (18, '0')
    let v7 : System.Guid = System.Guid $"{v6.[0..7]}-{v6.[8..11]}-{v6.[12..15]}-{v6.[16..17]}{v3.[21..]}"
    let v8 : (System.Guid -> string) = _.ToString()
    let v9 : string = v8 v7
    let v10 : int64 = int64 $"{v9.[0..7]}{v9.[9..12]}{v9.[14..17]}{v9.[19..20]}"
    let v11 : int64 = (v10 - System.DateTime.UtcNow.Ticks) / 100000L
    let v12 : string = $"%A{v11}"
    System.Console.WriteLine v12
    let v13 : bool = v11 = 0L
    let v15 : bool =
        if v13 then
            true
        else
            method1(v13)
    let v16 : string = $"__expect / actual: %A{v11} / expected: %A{0L}"
    let v17 : bool = v15 = false
 

In [ ]:
//// test

type DateTimeWithZone = {
    DateTime: System.DateTimeOffset
    TimeZone: System.TimeZoneInfo
}

try
    let now = System.DateTimeOffset.Now
    let timeZoneInfo = System.TimeZoneInfo.Local
    let dateTimeWithZone = { DateTime = now; TimeZone = timeZoneInfo }

    printfn "DateTime: %O" dateTimeWithZone.DateTime
    printfn "Time Zone: %s" dateTimeWithZone.TimeZone.DisplayName
    printfn "Is DST: %b" (timeZoneInfo.IsDaylightSavingTime(dateTimeWithZone.DateTime.DateTime))
    printfn "v1: %s" (dateTimeWithZone.TimeZone.GetUtcOffset(now) |> string)
    printfn "v2: %s" (dateTimeWithZone.TimeZone |> string)
with ex ->
    printfn "error: %A" ex

DateTime: 03/30/2024 21:53:44 +00:00
Time Zone: (UTC) Coordinated Universal Time
Is DST: false
v1: 00:00:00
v2: (UTC) Coordinated Universal Time


## main

In [ ]:
inl main () =
    $"let date_time_guid_from_date_time x = !date_time_guid_from_date_time x" : ()
    $"let date_time_from_guid x = !date_time_from_guid x" : ()
    $"let ticks_guid_from_ticks x = !ticks_guid_from_ticks x" : ()
    $"let ticks_from_guid x = !ticks_from_guid x" : ()
    $"let new_guid_from_date_time x = !new_guid_from_date_time x" : ()
    $"let new_guid_from_ticks x = !new_guid_from_ticks x" : ()

let rec closure1 (v0 : System.Guid) (v1 : System.DateTime) : System.Guid =
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v0
    let v4 : string = v1.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    let v5 : System.Guid = System.Guid $"{v4}{v3.[v4.Length..]}"
    v5
and closure0 () (v0 : System.Guid) : (System.DateTime -> System.Guid) =
    closure1(v0)
and closure5 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure4 (v0 : string) (v1 : string) : (string -> string) =
    closure5(v0, v1)
and closure3 () (v0 : string) : (string -> (string -> string)) =
    closure4(v0)
and method0 () : (string -> (string -> (string -> string))) =
    closure3()
and closure2 () (v0 : System.Guid) : System.DateTime =
    let v1 : (System.Guid -> string) = _.ToString()
    let v2 : string = v1 v0
    let v3 : (string -> (string -> (string -> string))) = method0()
    let mutable result = None
    #if FABLE_COMPILER_RUST && 